<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_10_28_Loading_and_Saving_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Saving Data in Spark

Collab Only code:

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!rm -f spark-3.2.0-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

#Install findspark using pip to make pyspark importable as regular library
!pip -q install findspark
import findspark
findspark.init()

Reading a text file textFile() in Python

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

lines = sc.textFile("spark-3.2.0-bin-hadoop3.2/README.md")
lines.collect()

['# Apache Spark',
 '',
 'Spark is a unified analytics engine for large-scale data processing. It provides',
 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a',
 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 'MLlib for machine learning, GraphX for graph processing,',
 'and Structured Streaming for stream processing.',
 '',
 '<https://spark.apache.org/>',
 '',
 '[![GitHub Action Build](https://github.com/apache/spark/actions/workflows/build_and_test.yml/badge.svg?branch=master)](https://github.com/apache/spark/actions/workflows/build_and_test.yml?query=branch%3Amaster)',
 '[![Jenkins Build](https://amplab.cs.berkeley.edu/jenkins/job/spark-master-test-sbt-hadoop-3.2/badge/icon)](https://amplab.cs.berkeley.edu/jenkins/job/spark-master-test-sbt-hadoop-3.2)',
 '[![AppVeyor Build](https://img.shields.io/appveyor/ci/ApacheSoftwareFoundation/spark/master.sv

Loading all the .md files in one directory

In [ ]:
input = sc.textFile("spark-3.2.0-bin-hadoop3.2/*.md")

---
**Load file in Google Colab:**

In [ ]:
from google.colab import files
files.upload()

Saving 2021-10-28-Loading_and_Saving_Data.ipynb to 2021-10-28-Loading_and_Saving_Data.ipynb


{'2021-10-28-Loading_and_Saving_Data.ipynb': b'{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"name":"2021-10-28-Loading_and_Saving_Data.ipynb","provenance":[{"file_id":"1BQpDSrMYqPRxWqJisQJHqCmjWzCGFg_X","timestamp":1635367621926}],"collapsed_sections":[]},"kernelspec":{"name":"python3","display_name":"Python 3"}},"cells":[{"cell_type":"markdown","metadata":{"id":"ZD1zvV9JtRsN"},"source":["# Loading and Saving Data in Spark"]},{"cell_type":"markdown","metadata":{"id":"BXcc49lmUYgz"},"source":["Collab Only code:"]},{"cell_type":"code","metadata":{"id":"L_U5TbtAUX_r"},"source":["!apt-get install openjdk-11-jdk-headless -qq > /dev/null\\n","!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz\\n","!tar xf spark-3.2.0-bin-hadoop3.2.tgz\\n","!rm -f spark-3.2.0-bin-hadoop3.2.tgz\\n","\\n","import os\\n","os.environ[\\"JAVA_HOME\\"] = \\"/usr/lib/jvm/java-11-openjdk-amd64\\"\\n","os.environ[\\"SPARK_HOME\\"] = \\"/content/spark-3.2.0-bin-hadoop3.2\\

In [ ]:
!ls -la

total 28
drwxr-xr-x  1 root root 4096 Oct 28 09:00 .
drwxr-xr-x  1 root root 4096 Oct 28 08:51 ..
-rw-r--r--  1 root root 5529 Oct 28 09:00 2021-10-28-Loading_and_Saving_Data.ipynb
drwxr-xr-x  4 root root 4096 Oct 26 13:33 .config
drwxr-xr-x  1 root root 4096 Oct 26 13:34 sample_data
drwxr-xr-x 13 1000 1000 4096 Oct  6 13:18 spark-3.2.0-bin-hadoop3.2


In [ ]:
sc.textFile("2021-10-28-Loading_and_Saving_Data.ipynb").collect()

['{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"name":"2021-10-28-Loading_and_Saving_Data.ipynb","provenance":[{"file_id":"1BQpDSrMYqPRxWqJisQJHqCmjWzCGFg_X","timestamp":1635367621926}],"collapsed_sections":[]},"kernelspec":{"name":"python3","display_name":"Python 3"}},"cells":[{"cell_type":"markdown","metadata":{"id":"ZD1zvV9JtRsN"},"source":["# Loading and Saving Data in Spark"]},{"cell_type":"markdown","metadata":{"id":"BXcc49lmUYgz"},"source":["Collab Only code:"]},{"cell_type":"code","metadata":{"id":"L_U5TbtAUX_r"},"source":["!apt-get install openjdk-11-jdk-headless -qq > /dev/null\\n","!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz\\n","!tar xf spark-3.2.0-bin-hadoop3.2.tgz\\n","!rm -f spark-3.2.0-bin-hadoop3.2.tgz\\n","\\n","import os\\n","os.environ[\\"JAVA_HOME\\"] = \\"/usr/lib/jvm/java-11-openjdk-amd64\\"\\n","os.environ[\\"SPARK_HOME\\"] = \\"/content/spark-3.2.0-bin-hadoop3.2\\"\\n","\\n","#Install findspark using pip to 

---

##Open**Data** Regione Siciliana
https://dati.regione.sicilia.it/dataset

####Load CSV from URL


In [ ]:
import requests

url ='https://dati.regione.sicilia.it/dataset/57983370-d035-4dd2-9ea1-b9552b4c7fb7/resource/cbd34026-25a5-489f-a57d-9cfbd57b63cf/download/56.csv'

file_name = url.split('/')[-1]
print(file_name)
r = requests.get(url, allow_redirects=True)
open(file_name,"wb").write(r.content)
!ls -la

56.csv
total 148
drwxr-xr-x  1 root root   4096 Oct 28 09:15 .
drwxr-xr-x  1 root root   4096 Oct 28 08:51 ..
-rw-r--r--  1 root root   5529 Oct 28 09:00 2021-10-28-Loading_and_Saving_Data.ipynb
-rw-r--r--  1 root root 119493 Oct 28 09:15 56.csv
drwxr-xr-x  4 root root   4096 Oct 26 13:33 .config
drwxr-xr-x  1 root root   4096 Oct 26 13:34 sample_data
drwxr-xr-x 13 1000 1000   4096 Oct  6 13:18 spark-3.2.0-bin-hadoop3.2


In [ ]:
sc.textFile("56.csv").collect()

In [ ]:
import requests

url = 'https://dati.regione.sicilia.it/download/dataset/arpa-qualita-aria-2019/filesystem/arpa-qualita-aria-2019-10_csv.zip'  
file_name = url.split('/')[-1]
r = requests.get(url, allow_redirects=True)
open(file_name,"wb").write(r.content)
print(file_name)
os.system("unzip "+file_name)
!rm -f *.zip; ls -la

arpa-qualita-aria-2019-10_csv.zip
total 5272
drwxr-xr-x  1 root root    4096 Oct 28 09:16 .
drwxr-xr-x  1 root root    4096 Oct 28 08:51 ..
-rw-r--r--  1 root root    5529 Oct 28 09:00 2021-10-28-Loading_and_Saving_Data.ipynb
-rw-r--r--  1 root root  119493 Oct 28 09:15 56.csv
-rw-r--r--  1 root root 5246127 Aug 12 11:20 arpa-qualita-aria-2019-10_csv.csv
drwxr-xr-x  4 root root    4096 Oct 26 13:33 .config
drwxr-xr-x  1 root root    4096 Oct 26 13:34 sample_data
drwxr-xr-x 13 1000 1000    4096 Oct  6 13:18 spark-3.2.0-bin-hadoop3.2


In [ ]:
sc.textFile("arpa-qualita-aria-2019-10_csv.csv").collect()

###**Reading a CSV file into a DataFrame, filter some columns and save it**

In [ ]:
# Create DataFrame
df = spark.read.csv("arpa-qualita-aria-2019-10_csv.csv",sep=";",header=True,inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- stazione_id: integer (nullable = true)
 |-- periodo_media: string (nullable = true)
 |-- inquinante_id: integer (nullable = true)
 |-- misura_valore: double (nullable = true)
 |-- misura_dataora: timestamp (nullable = true)
 |-- misura_anno: integer (nullable = true)

+-----------+-------------+-------------+-------------+-------------------+-----------+
|stazione_id|periodo_media|inquinante_id|misura_valore|     misura_dataora|misura_anno|
+-----------+-------------+-------------+-------------+-------------------+-----------+
|    1908498|            h|           10|     0.152034|2019-01-01 00:00:00|       2019|
|    1908498|            h|           10|     0.158291|2019-01-01 01:00:00|       2019|
|    1908498|            h|           10|     0.159747|2019-01-01 02:00:00|       2019|
|    1908498|            h|           10|     0.156639|2019-01-01 04:00:00|       2019|
|    1908498|            h|           10|     0.140042|2019-01-01 05:00:00|       2019|
+-----------+----

Filter data by several columns

In [ ]:
dataF=df.select("stazione_id","misura_dataora","misura_valore")
dataF.show()

+-----------+-------------------+-------------+
|stazione_id|     misura_dataora|misura_valore|
+-----------+-------------------+-------------+
|    1908498|2019-01-01 00:00:00|     0.152034|
|    1908498|2019-01-01 01:00:00|     0.158291|
|    1908498|2019-01-01 02:00:00|     0.159747|
|    1908498|2019-01-01 04:00:00|     0.156639|
|    1908498|2019-01-01 05:00:00|     0.140042|
|    1908498|2019-01-01 06:00:00|     0.130085|
|    1908498|2019-01-01 07:00:00|     0.131141|
|    1908498|2019-01-01 08:00:00|     0.130212|
|    1908498|2019-01-01 09:00:00|     0.176405|
|    1908498|2019-01-01 10:00:00|     0.164526|
|    1908498|2019-01-01 11:00:00|     0.168561|
|    1908498|2019-01-01 12:00:00|     0.163831|
|    1908498|2019-01-01 13:00:00|     0.206306|
|    1908498|2019-01-01 14:00:00|      0.15871|
|    1908498|2019-01-01 15:00:00|     0.129604|
|    1908498|2019-01-01 16:00:00|     0.139725|
|    1908498|2019-01-01 17:00:00|     0.159513|
|    1908498|2019-01-01 18:00:00|     0.

Save only the filtered Data

In [ ]:
dataF.write.csv("dati_filtrati.csv",mode='overwrite',header=True)

In [ ]:
!ls -la

In [ ]:
sc.textFile("dati_filtrati.csv").collect()

Let's read this new file back into an RDD

In [ ]:
data = spark.read.csv("dati_filtrati.csv",header=True,inferSchema=True)
data.printSchema()
data.show()

root
 |-- stazione_id: integer (nullable = true)
 |-- misura_dataora: timestamp (nullable = true)
 |-- misura_valore: double (nullable = true)

+-----------+-------------------+-------------+
|stazione_id|     misura_dataora|misura_valore|
+-----------+-------------------+-------------+
|    1908498|2019-01-01 00:00:00|     0.152034|
|    1908498|2019-01-01 01:00:00|     0.158291|
|    1908498|2019-01-01 02:00:00|     0.159747|
|    1908498|2019-01-01 04:00:00|     0.156639|
|    1908498|2019-01-01 05:00:00|     0.140042|
|    1908498|2019-01-01 06:00:00|     0.130085|
|    1908498|2019-01-01 07:00:00|     0.131141|
|    1908498|2019-01-01 08:00:00|     0.130212|
|    1908498|2019-01-01 09:00:00|     0.176405|
|    1908498|2019-01-01 10:00:00|     0.164526|
|    1908498|2019-01-01 11:00:00|     0.168561|
|    1908498|2019-01-01 12:00:00|     0.163831|
|    1908498|2019-01-01 13:00:00|     0.206306|
|    1908498|2019-01-01 14:00:00|      0.15871|
|    1908498|2019-01-01 15:00:00|     0.

In [ ]:
data.createOrReplaceTempView("table01")
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  table01|       true|
+---------+---------+-----------+



# **Hive Example**

Using Hive to create and read a table - Simple Example

In [ ]:
from pyspark.sql import HiveContext, Row

hiveCtx = HiveContext(sc)
ex1 = hiveCtx.read.csv("dati_filtrati.csv",header=True,inferSchema=True)

ex1.registerTempTable("Table02")
results = hiveCtx.sql("SELECT * FROM table02").show()

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:604: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  FutureWarning
/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+-----------+-------------------+-------------+
|stazione_id|     misura_dataora|misura_valore|
+-----------+-------------------+-------------+
|    1908498|2019-01-01 00:00:00|     0.152034|
|    1908498|2019-01-01 01:00:00|     0.158291|
|    1908498|2019-01-01 02:00:00|     0.159747|
|    1908498|2019-01-01 04:00:00|     0.156639|
|    1908498|2019-01-01 05:00:00|     0.140042|
|    1908498|2019-01-01 06:00:00|     0.130085|
|    1908498|2019-01-01 07:00:00|     0.131141|
|    1908498|2019-01-01 08:00:00|     0.130212|
|    1908498|2019-01-01 09:00:00|     0.176405|
|    1908498|2019-01-01 10:00:00|     0.164526|
|    1908498|2019-01-01 11:00:00|     0.168561|
|    1908498|2019-01-01 12:00:00|     0.163831|
|    1908498|2019-01-01 13:00:00|     0.206306|
|    1908498|2019-01-01 14:00:00|      0.15871|
|    1908498|2019-01-01 15:00:00|     0.129604|
|    1908498|2019-01-01 16:00:00|     0.139725|
|    1908498|2019-01-01 17:00:00|     0.159513|
|    1908498|2019-01-01 18:00:00|     0.

In [ ]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  table01|       true|
|         |  table02|       true|
+---------+---------+-----------+



In [ ]:
sql = spark.sql

sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  table01|       true|
|         |  table02|       true|
+---------+---------+-----------+



#Spark SQL

(https://spark.apache.org/docs/latest/sql-programming-guide.html)

**Reference**

(https://spark.apache.org/docs/latest/sql-ref.html)